In [1]:
import pickle
from pathlib import Path
from datetime import datetime

import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster, grid_search_forecaster

from catboost import CatBoostRegressor

SEED = 0

In [2]:
tickers = [file.stem for file in Path('oil_gas_train_test').iterdir()]

In [3]:
name2row = pd.Series({
    'Revenue': 30,
    'Gross Profit': 35,
    'Operating Income': 44,
    'Pretax Income': 54,
    'Net Income Avail to Common': 69,
    'Diluted EPS': 78,
    'Total Assets': 167,
    'Total Liabilities & Equity': 212,
    'Cash from Operations': 277,
    'Cash from Investing Activities': 295,
    'Cash from Financing Activities': 307,
    'Shareholder Equity Net Income': 328,
    'EBITDA / Interest': 353,
    'EBITDA - CAPEX / Interest': 354,
    'CAPEX / REVENUE': 355,
    'CAPEX / EBITDA': 356,
    'FCF / DEBT': 356,
    'Gross Leverage (Total Debt / EBITDA)': 358,
    'Net Leverage (Net Debt / EBITDA)': 359,
    'Total Liabilities / Tangible Assets': 360,
    'Cash + AR / Total Assets': 361,
    'Cash % of Revenue': 362    
})

In [4]:
def load_data(ticker, path_template='oil_gas_train_test/{}.xlsx'):
    path = path_template.format(ticker)
    data = pd.read_excel(
        path,
        engine='openpyxl',
        usecols=range(9, 51),
        header=28,
        na_values=['n.a.', '-']
    )
    data = data.reindex(name2row - 30 )
    data = data.set_index(name2row.index).T
    data = data.dropna(axis=1, how='all')
    data = data.astype(np.float32)
    data = data.asfreq('3M')
    quarter = pd.Series(data.index).dt.quarter
    data['quarter_cos'] = np.cos(quarter - 1 / 4 * 2 * np.pi)
    data['quarter_sin'] = np.sin(quarter - 1 / 4 * 2 * np.pi)
    return data

In [5]:
data = {ticker: load_data(ticker) for ticker in tickers}

In [6]:
def prepare_X_y(df, column):
    X = df.drop(columns=[column])
    y = df[column]
    change = y.pct_change()
    max_idx = change.argmax()
    if change.iloc[max_idx] == np.inf:
        X = X.iloc[max_idx:]
        y = y.iloc[max_idx:]
    X = (X - X.mean()) / X.std()
    X = X.fillna(method='bfill')
    X = X.dropna(how='all', axis=1)
    X = X.fillna(method='ffill')
    y = y.fillna(method='ffill').fillna(method='bfill')

    return X, y

In [7]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'depth': [3, 4, 5, 6],
    'learning_rate': [0.1, 0.05, 0.01],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}

metrics = {
    'MAE': mean_absolute_error,
    'MAPE': mean_absolute_percentage_error,
    'R2': r2_score
}

In [8]:
LAGS = [1, 2, 3, 4]
END_TRAIN = datetime(2020, 6, 30)


def train_model(ticker, target_col):
    X, y = prepare_X_y(data[ticker], target_col)

    forecaster = ForecasterAutoreg(
        regressor=CatBoostRegressor(random_state=SEED, loss_function='MAPE', logging_level='Silent'),
        lags=LAGS
    )
    _ = grid_search_forecaster(
        forecaster=forecaster,
        y=y.loc[:END_TRAIN],
        exog=X.loc[:END_TRAIN],
        param_grid=param_grid,
        fixed_train_size=False,
        steps=4,
        refit=True,
        metric='mean_absolute_percentage_error',
        initial_train_size=len(y.loc[:END_TRAIN])-12,
        return_best=True,
        verbose=False
    )

    def backtest(metric):
        return backtesting_forecaster(
            forecaster=forecaster,
            y=y,
            exog=X,
            initial_train_size=len(y.loc[:END_TRAIN]),
            steps=4,
            refit=True,
            metric=metric,
            verbose=False
        )

    scores = dict()
    for scoring_label, scoring_function in metrics.items():
        if scoring_label != 'MAPE':
            scores[scoring_label] = backtest(scoring_function)[0]
        else:
            scores[scoring_label], predictions = backtest(scoring_function)
            predictions = predictions['pred']

    return forecaster, scores, predictions

In [9]:
revenue = dict()
for ticker in tickers:
    try:
        model, scores, _ = train_model(ticker, 'Revenue')
        with open(f'models/revenue/{ticker}.pickle', 'wb') as f:
            pickle.dump(model, f)
        revenue[ticker] = scores
    except:
        continue

Number of models compared: 180.


loop param_grid:  47%|████████████████▌                  | 85/180 [00:23<00:24,  3.91it/s]


loop param_grid:  96%|████████████████████████████████▍ | 172/180 [00:51<00:02,  3.11it/s]


loop lags_grid: 100%|███████████████████████████████████████| 1/1 [00:54<00:00, 54.40s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4] 
  Parameters: {'depth': 6, 'l2_leaf_reg': 3, 'learning_rate': 0.05, 'n_estimators': 150}
  Backtesting metric: 0.6216965858464639

Number of models compared: 180.


...

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4] 
  Parameters: {'depth': 4, 'l2_leaf_reg': 9, 'learning_rate': 0.1, 'n_estimators': 150}
  Backtesting metric: 0.554818334564522

Number of models compared: 180.


loop param_grid:  47%|████████████████▌                  | 85/180 [00:29<00:29,  3.26it/s]


loop param_grid:  96%|████████████████████████████████▍ | 172/180 [01:00<00:02,  3.18it/s]


loop lags_grid: 100%|███████████████████████████████████████| 1/1 [01:03<00:00, 63.21s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4] 
  Parameters: {'depth': 3, 'l2_leaf_reg': 3, 'learning_rate': 0.05, 'n_estimators': 150}
  Backtesting metric: 0.41780233291493013



In [10]:
pd.DataFrame(revenue).T

,MAE,MAPE,R2
DVN,349.153939,0.265442,0.309428
MTDR,72.675528,0.223164,-0.515267
MCF,22.891410,0.363146,-0.752833
SWN,104.664584,0.103159,0.444033
XEC,61.855299,0.106506,0.409778
BSM,21.768962,0.400403,-2.640084
PVAC,14.727340,0.133769,0.069545
BATL,6.264977,0.126179,0.311493
AMPY,3.853258,0.051855,0.781246
CHK,1041.709300,1.129520,-26.289142


In [14]:
profit = dict()
for ticker in tickers:
    try:
        model, scores, _ = train_model(ticker, 'Gross Profit')
        with open(f'models/profit/{ticker}.pickle', 'wb') as f:
            pickle.dump(model, f)
        profit[ticker] = scores
    except:
        continue

Number of models compared: 180.


loop param_grid:  47%|████████████████▌                  | 85/180 [00:23<00:26,  3.56it/s]


loop param_grid:  96%|████████████████████████████████▍ | 172/180 [00:51<00:02,  2.84it/s]


loop lags_grid: 100%|███████████████████████████████████████| 1/1 [00:53<00:00, 53.92s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4] 
  Parameters: {'depth': 5, 'l2_leaf_reg': 1, 'learning_rate': 0.1, 'n_estimators': 100}
  Backtesting metric: 0.2098198257769849

Number of models compared: 180.


...

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4] 
  Parameters: {'depth': 4, 'l2_leaf_reg': 3, 'learning_rate': 0.1, 'n_estimators': 150}
  Backtesting metric: 2.0643779900186963

Number of models compared: 180.


loop param_grid:  47%|████████████████▌                  | 85/180 [00:33<00:36,  2.59it/s]


loop param_grid:  96%|████████████████████████████████▍ | 172/180 [01:09<00:03,  2.43it/s]


loop lags_grid: 100%|███████████████████████████████████████| 1/1 [01:13<00:00, 73.08s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4] 
  Parameters: {'depth': 6, 'l2_leaf_reg': 3, 'learning_rate': 0.1, 'n_estimators': 50}
  Backtesting metric: 0.6843159634872468



In [15]:
pd.DataFrame(profit).T

,MAE,MAPE,R2
DVN,95.725975,8.869638e-01,0.307318
MTDR,55.706970,3.186512e-01,-0.396698
MCF,13.877948,6.463081e-01,-1.166226
SWN,117.913135,2.179811e-01,-0.416519
XEC,74.306057,2.805937e-01,0.067876
BSM,19.909301,1.140529e+00,-2.615945
BATL,12.782279,3.594201e-01,-0.872784
AMPY,17.979828,6.666050e-01,-1.845375
CHK,254.639379,7.871637e-01,-0.331188
OVV,134.573101,1.979196e+00,0.140084
